In [1]:
# Load the MergedSummary CSV created at the end of Project 3
import pandas as pd

merged_summary = pd.read_csv('MergedSummary.csv')

# Group by state (region) and calculate the number of observations for each chain
state_observations = merged_summary.groupby('region').agg(
    Focal_Chain_Observations=('SumDailyVisits_Focal', 'count'),
    Benchmark_Chain_Observations=('SumDailyVisits_Benchmark', 'count')
).reset_index()

# Display the table
print("Number of observations for each state:")
print(state_observations)


Number of observations for each state:
   region  Focal_Chain_Observations  Benchmark_Chain_Observations
0      AZ                      1813                          1813
1      CA                       917                           917
2      CO                      1827                          1827
3      CT                       294                           294
4      DC                       938                           938
5      DE                      1827                          1827
6      FL                      1827                          1827
7      GA                      1204                          1204
8      IL                      1827                          1827
9      IN                      1827                          1827
10     KS                      1771                          1771
11     KY                      1827                          1827
12     LA                       518                           518
13     MA                      1827  

The observations are the same since the merged set from Project 3 is what is inclusive of both, Qdoba having data only in 33 states, so we use the lower number (Chipotle is in 46)

1.a

In [2]:
import pandas as pd

# Load the required datasets
merged_summary = pd.read_csv('MergedSummary.csv')  # This contains state-level aggregated data
census_demographics = pd.read_csv('census_state_2016.csv')  # This contains state-level demographic data

# Perform a state-level merge
final_analysis_sample = pd.merge(
    merged_summary,
    census_demographics,
    left_on='region',        # State in MergedSummary
    right_on='state_abbr',   # State abbreviation in Census data
    how='inner'
)

# Ensure we group by state-level observations
final_analysis_sample_grouped = final_analysis_sample.groupby('region').first().reset_index()

# Save the final dataset for further analysis
final_analysis_sample_grouped.to_csv('Final_Analysis_Sample.csv', index=False)
print("Final state-level dataset saved as 'Final_Analysis_Sample.csv'.")


Final state-level dataset saved as 'Final_Analysis_Sample.csv'.


1.b 
Number of observations in MergedSummary: 34 (unique states)
Number of observations in Census_demographics: 52 (unique states)
Number of matched observations in Final_analysis_sample: 34 (states)
Number of observations in MergedSummary but not in Census_demographics: 0
Number of observations in Census_demographics but not in MergedSummary: 18

The imperfect matches come from the fact that Chipotle only really operates in 34 unique places that are in conjunction with the benchmark Qdoba, the census data has information from all the states so its ging to be a larger number.


In [3]:

import numpy as np


# Ensure the 'new_date' column is in datetime format for extracting day of the week
final_analysis_sample['new_date'] = pd.to_datetime(final_analysis_sample['new_date'])

# Create Day of Week Indicators
final_analysis_sample['day_of_week'] = final_analysis_sample['new_date'].dt.day_name()
day_dummies = pd.get_dummies(final_analysis_sample['day_of_week'], prefix='day', drop_first=True)

# Include 'BlueState' Variable (already defined in Project 3)
blue_states = ['CA', 'NY', 'WA', 'MA', 'MD', 'IL', 'NJ', 'VT', 'RI', 'CT', 'DE', 'HI', 'OR', 'NM', 'NV']
final_analysis_sample['BlueState'] = final_analysis_sample['region'].apply(lambda x: 1 if x in blue_states else 0)

# Log Transform the Population and Hispanic/Latino Variables
final_analysis_sample['LogTotalPopulation'] = np.log(final_analysis_sample['total_population'] + 1)
final_analysis_sample['LogHispanicLatino'] = np.log(final_analysis_sample['hispanic_latino'] + 1)

# Add the day-of-week dummies to the dataset
final_analysis_sample = pd.concat([final_analysis_sample, day_dummies], axis=1)

# Save the updated dataset for further analysis
final_analysis_sample.to_csv('Updated_Final_Analysis_Sample.csv', index=False)

# Display a preview of the dataset
print("Updated Final Analysis Sample with Day-of-Week Indicators and Log Transforms:")
print(final_analysis_sample.head())


Updated Final Analysis Sample with Day-of-Week Indicators and Log Transforms:
  region   new_date  Nunits_Benchmark  SumDailyVisits_Benchmark  \
0     AZ 2018-01-01               1.0                       2.0   
1     AZ 2018-01-02               1.0                       2.0   
2     AZ 2018-01-03               1.0                      10.0   
3     AZ 2018-01-04               1.0                       4.0   
4     AZ 2018-01-05               1.0                       5.0   

   MaxDailyVisitsPerUnit_Benchmark  MinDailyVisitsPerUnit_Benchmark  \
0                              2.0                              2.0   
1                              2.0                              2.0   
2                             10.0                             10.0   
3                              4.0                              4.0   
4                              5.0                              5.0   

   MedDailyVisitsPerUnit_Benchmark  Nunits_Focal  SumDailyVisits_Focal  \
0                 

In [4]:
# Check correlation between LogHispanicLatino and BlueState
correlation = final_analysis_sample[['LogHispanicLatino', 'BlueState']].corr()

# Display the correlation matrix
print("Correlation between LogHispanicLatino and BlueState:")
print(correlation)


Correlation between LogHispanicLatino and BlueState:
                   LogHispanicLatino  BlueState
LogHispanicLatino            1.00000    0.22871
BlueState                    0.22871    1.00000


The correlation between Being Hispanic_latino is not that strong with Blue states so we are not too worried with using this in conjunction. We can see impact using VIF or tolerance later on in the regression part, but using Ridge and lasso can help mute this.

In [5]:
# List of numerical variables to summarize
numerical_vars = ['LogTotalPopulation', 'LogHispanicLatino', 'median_age', 'housing_units', 'total_population']

# Summarize numerical variables
numerical_summary = final_analysis_sample[numerical_vars].describe().T
numerical_summary['Median'] = final_analysis_sample[numerical_vars].median()
numerical_summary = numerical_summary.rename(columns={
    'count': 'Count', 'mean': 'Mean', 'std': 'Std Dev', 'min': 'Min', 'max': 'Max'
})[['Count', 'Mean', 'Median', 'Std Dev', 'Min', 'Max']]

print("Summary of Numerical Variables:")
print(numerical_summary)

# List of categorical variables to summarize
categorical_vars = ['BlueState']  # Add any new categorical variables to this list

# Summary of categorical variables
for var in categorical_vars:
    cat_summary = final_analysis_sample[var].value_counts().reset_index()
    cat_summary.columns = [var, 'Frequency']
    print(f"\nSummary of Categorical Variable ({var}):")
    print(cat_summary)

# Frequency distribution for day-of-week indicators
day_dummies = [col for col in final_analysis_sample.columns if col.startswith('day_')]
day_dummies_summary = final_analysis_sample[day_dummies].sum().reset_index()
day_dummies_summary.columns = ['Day of Week Indicator', 'Frequency']

print("\nSummary of Day-of-Week Indicators:")
print(day_dummies_summary)


Summary of Numerical Variables:
                      Count          Mean        Median       Std Dev  \
LogTotalPopulation  53186.0  1.562239e+01  1.570758e+01  8.514909e-01   
LogHispanicLatino   53186.0  1.337541e+01  1.328232e+01  1.295267e+00   
median_age          53186.0  3.817867e+01  3.840000e+01  1.763516e+00   
housing_units       53186.0  3.539664e+06  2.854595e+06  2.824860e+06   
total_population    53186.0  8.421578e+06  6.633053e+06  7.253647e+06   

                              Min           Max  
LogTotalPopulation      13.431569  1.748546e+01  
LogHispanicLatino       10.182444  1.654211e+01  
median_age              33.900000  4.230000e+01  
housing_units       313703.000000  1.406138e+07  
total_population    681170.000000  3.925002e+07  

Summary of Categorical Variable (BlueState):
   BlueState  Frequency
0          0      35532
1          1      17654

Summary of Day-of-Week Indicators:
  Day of Week Indicator                                          Frequency


1.c

Ended up adding housing_units, this is a proxy to how urban/developed the area might be, chains like Chipotle and Qdoba are more likely to be their. 

In [6]:
# Load the final dataset for analysis
final_analysis_sample = pd.read_csv('Updated_Final_Analysis_Sample.csv')

# Compute the correlation between 'total_population' and 'hispanic_latino'
correlation = final_analysis_sample[['total_population', 'hispanic_latino']].corr()

# Compute the ratio of 'hispanic_latino' to 'total_population'
final_analysis_sample['hispanic_to_population_ratio'] = (
    final_analysis_sample['hispanic_latino'] / final_analysis_sample['total_population']
)

# Summary statistics for the ratio
ratio_summary = final_analysis_sample['hispanic_to_population_ratio'].describe()

correlation, ratio_summary

(                  total_population  hispanic_latino
 total_population          1.000000         0.920528
 hispanic_latino           0.920528         1.000000,
 count    53186.000000
 mean         0.133215
 std          0.093905
 min          0.014627
 25%          0.066973
 50%          0.102951
 75%          0.189798
 max          0.390605
 Name: hispanic_to_population_ratio, dtype: float64)

In [7]:
# Calculate the proportion of Hispanic/Latino population for each state
final_analysis_sample['HispanicLatino_Proportion'] = (
    final_analysis_sample['hispanic_latino'] / final_analysis_sample['total_population']
)

# Group by state to calculate the average proportion for each state
state_hispanic_latino_proportion = final_analysis_sample.groupby('region')['HispanicLatino_Proportion'].mean().reset_index()

# Rename columns for clarity
state_hispanic_latino_proportion.columns = ['State', 'HispanicLatino_Proportion']

# Display the results
print(state_hispanic_latino_proportion)


   State  HispanicLatino_Proportion
0     AZ                   0.309444
1     CA                   0.389319
2     CO                   0.213195
3     CT                   0.157236
4     DC                   0.109256
5     DE                   0.091540
6     FL                   0.248713
7     GA                   0.093337
8     IL                   0.170198
9     IN                   0.067552
10    KS                   0.115884
11    KY                   0.034240
12    LA                   0.049377
13    MA                   0.114463
14    MD                   0.097521
15    MI                   0.049498
16    MN                   0.052258
17    MO                   0.039975
18    NC                   0.091764
19    ND                   0.034876
20    NE                   0.106155
21    NJ                   0.199751
22    NV                   0.284561
23    NY                   0.189798
24    OK                   0.102951
25    OR                   0.127659
26    PA                   0

Decided to remove total population, and just use hispanic_latino, they are higly correlated, but since the chain and benchmark chain is based around Mexican food, we are going to prioritize this var as it has alot of information that total population has and a potentially a bit more in this context.

In [27]:
# Import necessary libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load the updated dataset
data = pd.read_csv('Updated_Final_Analysis_Sample.csv')

# Drop any rows with missing values in the features or target variable
data = data.dropna(subset=[
    'LogHispanicLatino', 
    'BlueState', 
    'median_age',
    'housing_units',
    'total_population',
    'day_Monday', 
    'day_Tuesday', 
    'day_Wednesday', 
    'day_Thursday', 
    'day_Saturday',
    'SumDailyVisits_Focal',
    'SumDailyVisits_Benchmark'
])

# Compute the dependent variable (Y)
data['Y'] = np.log(data['SumDailyVisits_Focal'] + 1) - np.log(data['SumDailyVisits_Benchmark'] + 1)

# Define features used in the model
features = [
    'LogHispanicLatino', 
    'BlueState', 
    'median_age',
    'housing_units',
    'day_Monday', 
    'day_Tuesday', 
    'day_Wednesday', 
    'day_Thursday', 
    'day_Saturday'
]

# Select the features and target variable
X = data[features]
y = data['Y']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Generate summary statistics before scaling
print("Training Sample Summary Before Scaling:")
print(X_train.describe())

print("\nTest Sample Summary Before Scaling:")
print(X_test.describe())

# Standardize the features using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert the scaled data back to DataFrames
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=features)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=features)


Training Sample Summary Before Scaling:
       LogHispanicLatino     BlueState    median_age  housing_units
count       37230.000000  37230.000000  37230.000000   3.723000e+04
mean           13.381751      0.331319     38.177094   3.560557e+06
std             1.296656      0.470694      1.764395   2.838633e+06
min            10.182444      0.000000     33.900000   3.137030e+05
25%            12.727571      0.000000     36.700000   1.732887e+06
50%            13.282322      0.000000     38.400000   2.854595e+06
75%            13.982058      1.000000     39.400000   4.540697e+06
max            16.542106      1.000000     42.300000   1.406138e+07

Test Sample Summary Before Scaling:
       LogHispanicLatino     BlueState    median_age  housing_units
count       15956.000000  15956.000000  15956.000000   1.595600e+04
mean           13.360626      0.333354     38.182333   3.490916e+06
std             1.291940      0.471427      1.761513   2.791939e+06
min            10.182444      0.000000 

2.a 

Number of observations in the training sample: 37230
Number of observations in the test sample: 15956 

Housing Units Difference: The test sample has, on average, 69,641 fewer housing units than the training sample.
Total Population Difference: The test sample has, on average, 174,688 fewer people than the training sample.

The differnces overall for all the features are not quit significant just by looking at the standard deviation.

In [9]:
# Reset indices of X_train_scaled_df and y_train
X_train_scaled_df = X_train_scaled_df.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

# Add a constant term for the intercept to the training data
X_train_scaled_df = sm.add_constant(X_train_scaled_df)

# Fit the linear regression model using statsmodels on the training data
model = sm.OLS(y_train, X_train_scaled_df)
results = model.fit()

# Print the summary of the regression results
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.283
Model:                            OLS   Adj. R-squared:                  0.283
Method:                 Least Squares   F-statistic:                     1632.
Date:                Sun, 17 Nov 2024   Prob (F-statistic):               0.00
Time:                        21:10:36   Log-Likelihood:                -65058.
No. Observations:               37230   AIC:                         1.301e+05
Df Residuals:                   37220   BIC:                         1.302e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.5914      0.03

In [10]:
# Calculate the correlation coefficient
correlation = data['hispanic_latino'].corr(data['housing_units'])

print(f"Correlation between latinohispanic and housing_units: {correlation:.4f}")


Correlation between latinohispanic and housing_units: 0.8821


Re-splitting and running regression after removing housing_units, as its very correlated. Wanted to experiment where the variation explained from Housing_unit would go too, and it got observed into LogHispanicLatino.

In [80]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load the updated dataset
data = pd.read_csv('Updated_Final_Analysis_Sample.csv')

# Drop any rows with missing values in the features or target variable
data = data.dropna(subset=[
    'LogHispanicLatino', 
    'BlueState', 
    'median_age',
    'housing_units',
    'total_population',
    'day_Monday', 
    'day_Tuesday', 
    'day_Wednesday', 
    'day_Thursday', 
    'day_Saturday',
    'SumDailyVisits_Focal',
    'SumDailyVisits_Benchmark'
])

# Compute the dependent variable (Y)
data['Y'] = np.log(data['SumDailyVisits_Focal'] + 1) - np.log(data['SumDailyVisits_Benchmark'] + 1)

# Define features used in the model
features = [
    'LogHispanicLatino', 
    'BlueState', 
    'median_age',
    'day_Monday', 
    'day_Tuesday', 
    'day_Wednesday', 
    'day_Thursday', 
    'day_Saturday'
]

# Select the features and target variable
X = data[features]
y = data['Y']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Generate summary statistics before scaling
print("Training Sample Summary Before Scaling:")
print(X_train.describe())

print("\nTest Sample Summary Before Scaling:")
print(X_test.describe())

# Standardize the features using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert the scaled data back to DataFrames
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=features)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=features)


Training Sample Summary Before Scaling:
       LogHispanicLatino     BlueState    median_age
count       37230.000000  37230.000000  37230.000000
mean           13.381751      0.331319     38.177094
std             1.296656      0.470694      1.764395
min            10.182444      0.000000     33.900000
25%            12.727571      0.000000     36.700000
50%            13.282322      0.000000     38.400000
75%            13.982058      1.000000     39.400000
max            16.542106      1.000000     42.300000

Test Sample Summary Before Scaling:
       LogHispanicLatino     BlueState    median_age
count       15956.000000  15956.000000  15956.000000
mean           13.360626      0.333354     38.182333
std             1.291940      0.471427      1.761513
min            10.182444      0.000000     33.900000
25%            12.727571      0.000000     36.700000
50%            13.282322      0.000000     38.400000
75%            13.982058      1.000000     39.400000
max            16.5421

In [81]:
# Reset indices of X_train_scaled_df and y_train
X_train_scaled_df = X_train_scaled_df.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

# Add a constant term for the intercept to the training data
X_train_scaled_df = sm.add_constant(X_train_scaled_df)

# Fit the linear regression model using statsmodels on the training data
model = sm.OLS(y_train, X_train_scaled_df)
results = model.fit()

# Print the summary of the regression results
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.271
Model:                            OLS   Adj. R-squared:                  0.270
Method:                 Least Squares   F-statistic:                     1726.
Date:                Sun, 17 Nov 2024   Prob (F-statistic):               0.00
Time:                        22:29:15   Log-Likelihood:                -65378.
No. Observations:               37230   AIC:                         1.308e+05
Df Residuals:                   37221   BIC:                         1.309e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.9891      0.02

2.b
 
 R-squared:                       0.271
 Adj. R-squared:                  0.270
 
 We report low predictive power in this model being at rouhgly 0.270 (adjusted r^2)

    'LogHispanicLatino'is significant and has a positive effect
    'BlueState', is significant and has a positive effect
    'median_age', is significant and has a positive effect
    'housing_units', is significant and has a positive effect
    'day_Monday', is not  signifcant and has a negative effect
    'day_Tuesday', is not  signifcant and has a negative effect
    'day_Wednesday', is not  signifcant and has a negative effect
    'day_Thursday', is almost signifcant and has a negative effect
    'day_Saturday'is not  signifcant and has a positive effect

    It does make sense that Blue state has an effect, we saw it had an effect in project 3. HispanicLatino does make sense that it has a big positive effect considering these chains market themselves as "authentic" mexic food to some extent. HispanicLatino also has high correlation with total_population and Housing_units, which are significant if added to this, that variable has cofounding information (ie if their are more people, their will be more HispanicLatinos and housing_units) but it was a suprice by how much. Age made sense considering, younger people are in theory more likely to eat at these places.
    


In [64]:
from sklearn.linear_model import Lasso
import numpy as np

# List of alphas to try
alphas = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 5]

# Loop over the alphas
for alpha in alphas:
    linlasso = Lasso(alpha=alpha, max_iter=10000, random_state=42).fit(X_train_scaled, y_train)
    r2_train = linlasso.score(X_train_scaled, y_train)
    r2_test = linlasso.score(X_test_scaled, y_test)
    n_features = np.sum(linlasso.coef_ != 0)
    
    print('Alpha = {}\nFeatures kept: {}, R-squared (training): {:.3f}, R-squared (test): {:.3f}\n'
          .format(alpha, n_features, r2_train, r2_test))
    
    # Print the coefficients for this alpha
    print('Features with non-zero coefficients (sorted by absolute magnitude):')
    coef_list = list(zip(features, linlasso.coef_))
    non_zero_coefs = [e for e in coef_list if e[1] != 0]
    sorted_coefs = sorted(non_zero_coefs, key=lambda e: -abs(e[1]))
    for feature, coef in sorted_coefs:
        print('\t{}, {:.4f}'.format(feature, coef))
    print('\n' + '-'*60 + '\n')


Alpha = 1e-05
Features kept: 8, R-squared (training): 0.271, R-squared (test): 0.262

Features with non-zero coefficients (sorted by absolute magnitude):
	LogHispanicLatino, 3.7713
	median_age, 0.4764
	BlueState, 0.4531
	day_Thursday, -0.0395
	day_Monday, -0.0333
	day_Wednesday, -0.0286
	day_Saturday, 0.0259
	day_Tuesday, -0.0172

------------------------------------------------------------

Alpha = 0.0001
Features kept: 8, R-squared (training): 0.271, R-squared (test): 0.262

Features with non-zero coefficients (sorted by absolute magnitude):
	LogHispanicLatino, 3.7691
	median_age, 0.4743
	BlueState, 0.4531
	day_Thursday, -0.0379
	day_Monday, -0.0317
	day_Wednesday, -0.0270
	day_Saturday, 0.0262
	day_Tuesday, -0.0156

------------------------------------------------------------

Alpha = 0.001
Features kept: 7, R-squared (training): 0.270, R-squared (test): 0.261

Features with non-zero coefficients (sorted by absolute magnitude):
	LogHispanicLatino, 3.7467
	BlueState, 0.4532
	median_a

2.c The criteria I will use to select the model is the one most parsimonous (the best bang for its buck model). This helps to pick a more general, model that is less likely to overfit, save computational time and have varaibles that are as MECE as possible.

Picking Model Alpha = 0.01 with 3 features
LogHispanicLatino, 3.5232
	BlueState, 0.4547
	median_age, 0.2424
 These are the only NON-zero features, the other 5 features are 0, which where the dummy varaibles for the day of week. 

 The R^2 test is the the same as what we got for the Linear regression.



In [65]:
# ... (rest of your code)

# Standardize the features using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create DataFrames with original column names
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Train the Lasso model
lasso_model = Lasso(alpha=0.01, random_state=42)
lasso_model.fit(X_train_scaled_df, y_train)

# Calculate permutation importance
result = permutation_importance(
    lasso_model, X_test_scaled_df, y_test, n_repeats=10, random_state=42
)

# Get feature names and importance scores
feature_names = X_test_scaled_df.columns
importances = result.importances_mean

# Create a DataFrame for better visualization
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Print top 5 important features
print("Top 5 Important Features:")
print(importance_df.head(5))

Top 5 Important Features:
             Feature  Importance
0  LogHispanicLatino    0.405580
1          BlueState    0.036064
2         median_age    0.004231
3         day_Monday    0.000000
4        day_Tuesday    0.000000


2.c cont

Note to TA: I couldnt get !pip scikit-learn==1.2.2 to work, I tried in many ways but its something to do with my Mac. I used a different approach here

The results show that logHispanicLatino is the most importance compared to the rest of the variables. The next best 'Bluestate" is higher than the rest but very close to 0 so really the LogHispanicLatino is the most important.

In [66]:
# Define the penalty parameters to test
alphas = [0.1, 1, 10]

# Initialize results dictionary
ridge_results = {
    "Alpha": [],
    "Training R^2": [],
    "Test R^2": [],
    "Coefficients": []
}

# Loop through each alpha value and fit Ridge regression
for alpha in alphas:
    # Initialize Ridge regression with the current alpha
    ridge_model = Ridge(alpha=alpha)

    # Select the relevant features from the training and test sets
    X_train_subset = X_train[features]
    X_test_subset = X_test[features]

    # Fit the model on the training data
    ridge_model.fit(X_train_subset, y_train)

    # Predict Y for training and test sets
    y_train_pred = ridge_model.predict(X_train_subset)
    y_test_pred = ridge_model.predict(X_test_subset)

    # Calculate R-squared values
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    # Store results
    ridge_results["Alpha"].append(alpha)
    ridge_results["Training R^2"].append(train_r2)
    ridge_results["Test R^2"].append(test_r2)
    ridge_results["Coefficients"].append(ridge_model.coef_)

# Display results
print("Ridge Regression Results:")
for i, alpha in enumerate(ridge_results["Alpha"]):
    print(f"\nAlpha: {alpha}")
    print(f"Training R^2: {ridge_results['Training R^2'][i]:.4f}")
    print(f"Test R^2: {ridge_results['Test R^2'][i]:.4f}")
    print("Coefficients:")
    for feature, coef in zip(features, ridge_results["Coefficients"][i]):
        print(f"  {feature}: {coef:.4f}")

# Choose the best alpha (highest Test R^2)
best_alpha_idx = ridge_results["Test R^2"].index(max(ridge_results["Test R^2"]))
print("\nMost Appropriate Alpha:")
print(f"Alpha: {ridge_results['Alpha'][best_alpha_idx]}")
print(f"Test R^2: {ridge_results['Test R^2'][best_alpha_idx]:.4f}")

Ridge Regression Results:

Alpha: 0.1
Training R^2: 0.2705
Test R^2: 0.2616
Coefficients:
  LogHispanicLatino: 0.5930
  BlueState: 0.4531
  median_age: 0.0567
  day_Monday: -0.0335
  day_Tuesday: -0.0174
  day_Wednesday: -0.0288
  day_Thursday: -0.0397
  day_Saturday: 0.0258

Alpha: 1
Training R^2: 0.2705
Test R^2: 0.2616
Coefficients:
  LogHispanicLatino: 0.5930
  BlueState: 0.4530
  median_age: 0.0567
  day_Monday: -0.0335
  day_Tuesday: -0.0173
  day_Wednesday: -0.0288
  day_Thursday: -0.0396
  day_Saturday: 0.0258

Alpha: 10
Training R^2: 0.2705
Test R^2: 0.2616
Coefficients:
  LogHispanicLatino: 0.5930
  BlueState: 0.4525
  median_age: 0.0568
  day_Monday: -0.0333
  day_Tuesday: -0.0172
  day_Wednesday: -0.0287
  day_Thursday: -0.0395
  day_Saturday: 0.0259

Most Appropriate Alpha:
Alpha: 10
Test R^2: 0.2616


2.d

I chose Alpha is 10 as it was a "stricter" model, it hadthe best test R^2 and it help to lower potentilly inflation results from the logHispanicLatino variable that it could have from any correlation with housing_units.

     These features are very close to 0, similar to what we got using LASSO
      day_Monday: -0.0333
      day_Tuesday: -0.0172
      day_Wednesday: -0.0287
      day_Thursday: -0.0395
      day_Saturday: 0.0259

 These features had more relevant coefficnets, but we see a big change in the coefficiant of LogHispanicLatino. The value got lower, this could mean the this model was stricter to penalize overfitting of that variable.
  LogHispanicLatino: 0.5930
  BlueState: 0.4525
  median_age: 0.0568

  The signs and coefficient values where almsot idential to the LASSO results and it confirms previous conclusions.

  Test R^2: 0.2616 , it was slightly higher.

In [78]:

import warnings


# Suppress the warning
with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    result = permutation_importance(
        ridge_model, X_test_scaled, y_test, n_repeats=30, random_state=42
    )

# Get feature names and importance scores
feature_names = features
importances = result.importances_mean

# Create a DataFrame for better visualization
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Print top 5 important features
print("Top 5 Important Features:")
print(importance_df.head(5))

Top 5 Important Features:
             Feature  Importance
0  LogHispanicLatino    0.431775
1          BlueState    0.032096
2         median_age    0.007856
6       day_Thursday    0.000175
3         day_Monday    0.000145


2.d cont Our results are similar to LASSO permutations, but the levle of importance scaled down by about a 1/3 on LogHispanicLatino and housing_units. But both models are consistent in ranking LogHispanicLatino, then BlueState then Median age in terms of importance.

All 3 Models are good in figuring out which variables are signficant, but the linear regression may overstate the impact of LogHispanicLatino, and the use of penalty term helps reduce the variation inflation that may arise from that variable, and we see that the Ridge (alpha = 10) was the "Stricktest" in handled LogHispanicLatino.

